# Clustering algorithms

Taken from https://github.com/dashee87/blogScripts/blob/master/Jupyter/2017-05-09-Clustering-with-Scikit-with-GIFs.ipynb with only minor modifications, and also based on https://medium.com/@arifromadhan19/step-by-step-to-understanding-k-means-clustering-and-implementation-with-sklearn-b55803f519d6

## Overview

In this notebook we will discuss different clustering algorithms. Then we will see an example for the K-Means clustering, an unsupervised machine learning method.

# Libraries

In [ ]:
import warnings

import numpy  as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn import cluster, datasets, mixture

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors     import kneighbors_graph

from matminer.datasets                import load_dataset
from matminer.featurizers             import composition
from matminer.featurizers.base        import MultipleFeaturizer
from matminer.featurizers.conversions import StrToComposition

warnings.filterwarnings("ignore")

plt.rc('xtick', labelsize=18) 
plt.rc('ytick', labelsize=18)

blue   = '#0021A5'
orange = '#FA4616'

## 1. Generate sample data

In [ ]:
def cluster_plots(set1, set2, colors1=blue, colors2=blue, title1='Set 1',  title2='Set 2'):

    fig, ax = plt.subplots(1, 2, figsize=(16, 8), layout='tight')

    ax[0].set_title(title1,fontsize=20)
    ax[1].set_title(title2,fontsize=20)

    ax[0].set_xlim(min(set1[:,0]), max(set1[:,0]))
    ax[1].set_xlim(min(set2[:,0]), max(set2[:,0]))

    ax[0].set_ylim(min(set1[:,1]), max(set1[:,1]))
    ax[1].set_ylim(min(set2[:,1]), max(set2[:,1]))

    ax[0].scatter(set1[:,0], set1[:,1], s=8**2, lw=0, c=colors1)
    ax[1].scatter(set2[:,0], set2[:,1], s=8**2, lw=0, c=colors2)

    plt.show()

# Number of points
points = 1_000

# Define seed for reproducibility
np.random.seed(844)

clust1 = np.random.normal( 5, 2, (points,2))
clust2 = np.random.normal(15, 3, (points,2))

clust3 = np.random.multivariate_normal([17,3], [[1,0],[0,1]], points)
clust4 = np.random.multivariate_normal([2,16], [[1,0],[0,1]], points)

dataset1 = np.concatenate((clust1, clust2, clust3, clust4))

dataset2 = datasets.make_circles(n_samples=points, factor=.5, noise=.05)[0]

cluster_plots(dataset1, dataset2)

## 2. K-means clustering

K-means is likely one of the most popular clustering algorithms. The algorithm itself is relatively simple: Starting with a pre-specified number of cluster centroids, each data point is initally assigned to its nearest centre. In the next step, for each segment, the centres are moved to the centroid of the clustered points. The points are then reassigned to their nearest centre. The process is repeated until moving the centres derives little or no improvement (measured by the within cluster sum of squares- the total squared distance between each point and its cluster centre). The alogorithm is concisely illustrated by the GIF below.

![title](https://dashee87.github.io/images/kmeans.gif)

K-means clustering in scikit offers several extensions. To prevent the alogrithm returning sub-optimal clustering, the kmeans method includes the `n_init` and `method` parameters. The former just reruns the algorithm with n different initialisations and returns the best output (measured by the within cluster sum of squares). By setting the latter to 'kmeans++' (the default), the initial centres are smartly selected (i.e. better than random). This has the additional benefit of decreasing runtime (less steps to reach convergence).

Let's see a quick implementation of this algorithm for our dataset

In [ ]:
kmeans_dataset1 = cluster.KMeans(n_clusters=4, max_iter=300,
                                 init='k-means++', n_init=10)

kmeans_dataset2 = cluster.KMeans(n_clusters=2, max_iter=300,
                                 init='k-means++', n_init=10)

fit_kmeans_dataset1 = kmeans_dataset1.fit_predict(dataset1)
fit_kmeans_dataset2 = kmeans_dataset2.fit_predict(dataset2)

print('Set 1')
for idx, i in enumerate(np.unique(fit_kmeans_dataset1)):
    print(f'Cluster {idx} : {sum(fit_kmeans_dataset1==i)}')

cluster_plots(dataset1, dataset2, 
              colors1=fit_kmeans_dataset1,
              colors2=fit_kmeans_dataset2)

We can see that K-means performs well on Set 1, but fails for Set 2. In fact, these two datasets illustrate the strengths and weaknesses of the K-means method. The algorithm seeks and identifies globular (essentially spherical) clusters. If this assumption does not hold, the model output may be inadaquate (or just really bad). K-means also underperforms with clusters of different size and density. Let's test this assertion with a slighlty different set

In [ ]:
dataset3 = np.vstack( [dataset1[:2080,:],dataset1[3000:3080]] )
dataset4 = np.vstack( [dataset1[-2080:,],dataset1[:80]] )

kmeans_dataset1 = cluster.KMeans(n_clusters=4, max_iter=300, 
                                 init='k-means++',n_init=10)

kmeans_dataset2 = cluster.KMeans(n_clusters=4, max_iter=300, 
                                 init='k-means++',n_init=10)
                                                                                    

fit_kmeans_dataset1 = kmeans_dataset1.fit_predict(dataset3)
fit_kmeans_dataset2 = kmeans_dataset2.fit_predict(dataset4)

cluster_plots(dataset3, dataset4, 
              colors1=fit_kmeans_dataset1, colors2=fit_kmeans_dataset2,
              title1='Set 3', title2='Set 4')

For all its faults, the enduring popularity of k-means (and related algorithms) stems from its versatility. Its average complexity is $\mathcal{O}(k\,n\,T)$, where $k$, $n$ and $T$ are the number of clusters, samples and iterations, respectively. As such, it is considered one of the [fastest clustering algorithms](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html). And in the world of big data, this matters.

*If your boss wants 10 customer segments by close of business, then you'll probably use k-means and just hope no one knows the word [globular](https://www.merriam-webster.com/dictionary/globular).*

## 3. Expectation maximization

This technique is the application of the [general expectation maximisation (EM) algorithm](https://en.wikipedia.org/wiki/Expectation%E2%80%93maximization_algorithm) to the task of clustering. It is conceptually related and visually similar to k-means (see GIF below). Where k-means seeks to minimise the distance between the observations and their assigned centroids, EM estimates some latent variables (typically the mean and covariance matrix of a mutltinomial normal distribution (called [Gaussian Mixture Models (GMM)](http://scikit-learn.org/stable/modules/mixture.html))), so as to maximise the log-likelihood of the observed data. Similar to k-means, the algorithm converges to the final clustering by iteratively improving its performance (i.e. reducing the log-likelihood). However, again like k-means, there is no guarantee that the algorithm has settled on the global minimum rather than local minimum (a concern that increases in higher dimensions).

![title](https://dashee87.github.io/images/em_only.gif)

In contrast to K-means, observations are not explicitly assigned to clusters, but rather given probabilities of belonging to each distribution. If the underlying distribution is identified correctly (e.g. normal distribution in the GIF), then the algorithm performs well. In practice, especially for large datasets, the underlying distribution may not be retrievble, so EM clustering may not be well suited to such tasks.

Let's try this method with our Sets 1 and 2

In [ ]:
em_dataset1 = mixture.GaussianMixture(n_components=4,
                                      covariance_type='full')

em_dataset2 = mixture.GaussianMixture(n_components=2,
                                      covariance_type='full')

fit_em_dataset1 = em_dataset1.fit(dataset1)
fit_em_dataset2 = em_dataset2.fit(dataset2)

predict_em_dataset1 = fit_em_dataset1.predict(dataset1)
predict_em_dataset2 = fit_em_dataset2.predict(dataset2)

cluster_plots(dataset1, dataset2,
              colors1=predict_em_dataset1, 
              colors2=predict_em_dataset2)

No surprises here. EM clusters the first dataset perfectly, as the underlying data is normally distributed. In contrast, `Set 2` cannot be modelled accurately as a GMM, so that's why EM performs so poorly in this case.

## 4. Hierarchical clustering

Unlike K-means and EM, [hierarchical clustering](https://en.wikipedia.org/wiki/Hierarchical_clustering) (HC) does not require the user to specify the number of clusters beforehand. It instead returns an output (typically as a dendrogram- see GIF below), from which the user can decide the appropriate number of clusters (either manually or [algorithmically](https://joernhees.de/blog/2015/08/26/scipy-hierarchical-clustering-and-dendrogram-tutorial/)). If done manually, the user may cut the dendrogram where the merged clusters are too far apart (represented by a long lines in the dendrogram). Alternatively, the user can just return a specific number of clusters (similar to k-means).

![title](https://dashee87.github.io/images/hierarch.gif)

As its name suggests, it constructs a hierarchy of clusters based on proximity (e.g Euclidean distance or Manhattan distance- see GIF below). HC typically comes in two flavours (essentially, bottom up or top down): 

* Divisive: Starts with the entire dataset comprising one cluster that is iteratively split- one point at a time- until each point forms its own cluster.
* Agglomerative: The agglomerative method in reverse- individual points are iteratively combined until all points belong to the same cluster.

Another important concept in HC is the linkage criterion. This defines the distance between clusters as a function of the points in each cluster and determines which clusters are merged/split at each step. That clumsy sentence is neatly illustrated in the GIF below.

![title](https://dashee87.github.io/images/hierarch_1.gif)

In [ ]:
hc_dataset1 = cluster.AgglomerativeClustering(n_clusters=4, metric='euclidean', 
                                              linkage='ward')

hc_dataset2 = cluster.AgglomerativeClustering(n_clusters=2, metric='euclidean', 
                                              linkage='average')

fit_hc_dataset1 = hc_dataset1.fit_predict(dataset1)
fit_hc_dataset2 = hc_dataset2.fit_predict(dataset2)

print('Set 1')
for idx, i in enumerate(np.unique(fit_kmeans_dataset1)):
    print(f'Cluster {idx} : {sum(fit_kmeans_dataset1==i)}')

cluster_plots(dataset1, dataset2,
              colors1=fit_hc_dataset1,
              colors2=fit_hc_dataset2)

You may notice that HC does not perform so well on the noisy circles.

However, by imposing simple connectivity constraints (points can only cluster with their n(=5) nearest neighbours), HC can capture the non-globular structures within the dataset.

In [ ]:
hc_dataset2 = cluster.AgglomerativeClustering(n_clusters=2, metric='euclidean', 
                                              linkage='average')

fit_hc_dataset2 = hc_dataset2.fit_predict(dataset2)

# Connectivity constraints
connect = kneighbors_graph(dataset2, n_neighbors=5, include_self=False)

connect_hc_dataset2 = cluster.AgglomerativeClustering(n_clusters=2, metric='euclidean',
                                                      linkage='complete', connectivity=connect)

fit_connect_hc_dataset2 = connect_hc_dataset2.fit_predict(dataset2)

cluster_plots(dataset2, dataset2,
              colors1=fit_hc_dataset2,
              colors2=fit_connect_hc_dataset2,
              title1='Without Connectivity', title2='With Connectivity')

## 5. Example for K_means clustering

Before we apply the clustering technique, we create a dataset of materials with experimental bandgaps.

In [ ]:
data = load_dataset("expt_gap")

# Number of unique formulas
data.describe()

In [ ]:
# Sort by size of bandgap
data = data.sort_values('gap expt')

# Remove duplicate compositions
data = data.drop_duplicates('formula')

data.describe()

### 5.1 Obtain a Feature Vector for Each Material

The first step in building a machine learning model is to convert the raw materials data, here the composition, into the required input for an ML model: a finite list of quantitative attributes. Here we use the Magpie descriptors from Ward et al.

In [ ]:
features = [composition.Stoichiometry(), composition.ElementProperty.from_preset("magpie"),
            composition.ValenceOrbital(props=['avg']), composition.IonProperty(fast=True)]

feature_calculators = MultipleFeaturizer(features)

# Get the feature names
feature_labels = feature_calculators.feature_labels()

# Compute the features for all materials entries
data = StrToComposition(target_col_id='composition_obj').featurize_dataframe(data, 'formula')

data = feature_calculators.featurize_dataframe(data, col_id='composition_obj', ignore_errors=True)

print(f'Generated {len(feature_labels)} features')

print('Training set size: {} x {}'.format(*data[feature_labels].shape))

#print(f'Feature labels {feature_labels}')

In [ ]:
# Retain only numerical values
numerical_data = data.select_dtypes([np.number])

# Drop the columns that include incomplete data
numerical_data = numerical_data.dropna(axis=0)

numerical_data.describe()

### 5.2 Dimensionality reduction with principal component analysis

In [ ]:
# Standardizing the features
standardized_data = StandardScaler().fit_transform(numerical_data)

# Principal component analysis to project onto first two principal components
principal_component_analysis = PCA()
principal_component_analysis.fit(standardized_data)

# Plot explained variance
plt.figure( figsize=(8, 8) )

plt.plot(np.cumsum(principal_component_analysis.explained_variance_ratio_))

plt.xlabel('Number of components', fontsize=18)
plt.ylabel('Cumulative explained variance', fontsize=18)

plt.show()

> ### Assignment
>
> Compute the principal component analysis for 2 components. Save your results to a dataframe using `PC1` and `PC2` as the labels for the first and second principal component, respectively.
> You can use the sklearn function pca.
> Plot the data and color the data points by the experimental bandgap.

### 5.3 K-Means clustering

> ### Assignment
>
> Calculate the K-Means clustering of the `standardized_data` for a total of four clusters and plot your result. Color the points according to the cluster they belong too.